In [2]:
import re
import random
import pandas as pd
import hashlib

In [3]:
with open('certcards2.txt', mode='r', encoding='utf8') as f:
    all_cards = f.read()
    
card_split_pattern = r'\n\n\n\d+\n'
all_cards = re.split(card_split_pattern, all_cards)
# Use re.DOTALL to allow . to match newline characters
card_pattern = re.compile(r'(.+?)\n([\s\S]+)', re.DOTALL)
cards = [(match.group(1), match.group(2)) for cstring in all_cards if (match := re.search(card_pattern, cstring))]

# removing the cards that have no content and trimming
cards = [(subject, stripped_content) for subject, content in cards if len(stripped_content := content.strip()) > 5]

def hash_string_md5(s):
    """
    Hashes a string using MD5 and returns a truncated hash for efficiency.

    Parameters:
    - s (str): The input string to hash.

    Returns:
    - str: The truncated hexadecimal hash string.
    """
    if pd.isnull(s):
        return None  # Handle NaN values gracefully
    return hashlib.md5(s.encode('utf-8')).hexdigest()  # Truncate to first 12 characters


def remake_card_document(existing_cards: pd.DataFrame, filename: str='certcards2.txt'):
    with open(filename, mode='w', encoding='utf8') as f:
        i = 1
        for _, row in existing_cards.iterrows():
            print(i)
            f.write('\n'*6)
            f.write(str(i)+'\n')  
            f.write(row['head']+'\n')
            f.write(row['body'])
            i+=1
            # print(F"{row['head']}: {row['age']:.4f}")


existing_cards = pd.DataFrame(cards, columns=['head', 'body'])


# existing_cards['age'] = [random.random() for _ in existing_cards.index]
existing_cards['hash'] = existing_cards['body'].apply(hash_string_md5)
existing_cards

card_ages = pd.read_json('card_ages.json')
# found_cards = pd.DataFrame(cards, columns=['head', 'body'])
# found_cards['hash'] = found_cards['body'].apply(hash_string_md5)

cards_to_age = pd.merge(
    left=existing_cards,
    right=card_ages[['hash', 'age']],
    left_on='hash', right_on='hash',
    how='left'
)

cards_to_age['head'] = cards_to_age['head'].str.strip()

cards_to_age['age'] = cards_to_age['age'].fillna(0)
cards_to_age['age'] = cards_to_age['age'] * 1.05
cards_to_age['age'] = cards_to_age['age'] + [random.random() for _ in cards_to_age.index]

# cards_to_age = cards_to_age.sort_values('age', ascending=False)
cards_to_age.drop_duplicates(subset=['hash'], keep='first')
cards_to_age.to_json('card_ages.json', indent=2)

existing_cards = cards_to_age

In [4]:
existing_cards.head(20)

,head,body,hash,age
0,Python,The first bit of information that you can gath...,7da14bed0fb71ea27640010a678ff008,41.902089
1,Diffusers Library,Bayesian Inference\nThe process of updating pr...,efdb373cb8627f97e3f95fb18eddb4a6,41.620682
2,Diffusers Documentation,"Sometimes, old environment paths can linger in...",b7cd2d381a70974dd5c4efb35540e8b5,42.046512
3,MS Identity Platform,The redirect URIs to use in a desktop applicat...,eed9f81cb9d69a3dce61a716e3fa3cf9,41.793373
4,OAuth2.0,Access tokens are short lived. Refresh them af...,4f6d811c33c9f4c0a5649a91ec857826,41.341072
5,pandas,Control Over Record Path and Metadata in json_...,fb79a219774a3964029a13f87e447b8d,41.213640
6,DNS,• DNS Tunneling:\nA method of encapsulating da...,0a94b9586de2e120a12ae44db581156f,41.396708
7,Power BI,Creating POST Requests in Power Query: You can...,7f4b2da9dadb31749f50ef409cc33354,41.054293
8,pandas,Expanding List-Like Elements into Rows using e...,9818e2ebe36139ec05f56227ef5f4930,41.013047
9,Python,"PyTorch is based on Torch, a framework for doi...",b8e6d739eaf408b4ab9448ce48ef8036,41.111707


# Completely Random Shuffle

In [5]:
# rows, cols = existing_cards.shape

# existing_cards = existing_cards.sample(frac=1)
# remake_card_document(filename='certcards2.txt', existing_cards = existing_cards)



# Age Shuffle

In [6]:
# existing_cards = existing_cards.sort_values('age', ascending=False)
# remake_card_document(filename='certcards2.txt', existing_cards=existing_cards)

In [7]:
existing_cards[['head', 'age']].head(n = 10)

,head,age
0,Python,41.902089
1,Diffusers Library,41.620682
2,Diffusers Documentation,42.046512
3,MS Identity Platform,41.793373
4,OAuth2.0,41.341072
5,pandas,41.213640
6,DNS,41.396708
7,Power BI,41.054293
8,pandas,41.013047
9,Python,41.111707


# Headers with fewest notes first

In [8]:
# frequency = existing_cards['head'].value_counts(ascending=True)
# print(frequency)

# existing_cards = pd.merge(
#     left=existing_cards,
#     right=frequency.rename('frequency'),
#     left_on='head', right_index=True,
#     how='left'
# )
# existing_cards.sort_values(['frequency', 'head'], ascending=True, inplace=True)

# remake_card_document(filename='certcards2.txt', existing_cards=existing_cards)

In [9]:
existing_cards['head'].value_counts()

head
DNS                               36
Azure Kubernetes Learning Path    34
Python                            32
Diffusers Library                 29
Search Engine Optimization        29
sklearn                           21
Azure Functions                   19
OData                             17
MS Identity Platform              13
Power BI                          12
Kali Linux                        11
Diffusers from Hugging Face       10
Jane Eyre                         10
PP365                             10
Azure AI Search                    8
AKS                                8
pandas                             8
Dataverse Queries                  8
Conditional Access                 7
Workera.ai                         7
Dataverse Plugins                  7
General                            7
Diffusers Documentation            6
Git                                6
Azure VDI Project                  6
Azure OpenAI                       6
OData Requests                   

# Focus on one header

In [10]:
heads = existing_cards['head'].value_counts()
heads = heads[heads > 5].index.tolist()
one_header = random.sample(heads, 1)[0]

one_header = 'Python'
existing_cards['pick_head'] = existing_cards['head'].apply(
    lambda h: 0 if h == one_header else 1 
)

remake_card_document(existing_cards=existing_cards.sort_values('pick_head'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
existing_cards.sort_values('pick_head')['age'].head(10)

0      41.902089
18     40.224935
28     39.525599
30     39.461326
258    24.948932
267    23.362311
233    28.975078
239    28.245157
216    30.487140
217    30.673073
Name: age, dtype: float64